In [ ]:
from qampy import signals, impairments, equalisation, phaserec, helpers
import numpy as np
# here we use bokeh for plotting as it is much faster
from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, show
import warnings
warnings.filterwarnings('ignore')
output_notebook()

Generate the signal

In [ ]:
length = 10000

In [ ]:
sig = signals.SignalQAMGrayCoded(4, length, nmodes=2, fb=25e9)

In [ ]:
def plot_constellation(E):
    fig = figure(title="QPSK signal constellation", output_backend="webgl")
    fig.scatter(E[0].real, E[0].imag, color='red', alpha=0.3, legend="X")
    fig.scatter(E[1].real, E[1].imag, color='blue', alpha=0.3, legend="Y")
    fig.xaxis[0].axis_label = "In-Phase"
    fig.yaxis[0].axis_label = "Quadrature"
    show(fig)

In [ ]:
# plot_constellation(sig)

Resample and pulse shaping

In [ ]:
sigResam = sig.resample(sig.fb*2, beta=0.1, renormalise=True)
plot_constellation(sigResam)

Add linear impairments

In [ ]:
signoise = impairments.change_snr(sigResam, 20)
# plot_constellation(signoise)

In [ ]:
atten = 10 # unit:dB
signoise = signoise/10**0.5  # attenuation
# plot_constellation(signoise)

In [ ]:
signoise = impairments.apply_phase_noise(signoise, 10e3)
# plot_constellation(signoise)

In [ ]:
signoise = impairments.add_carrier_offset(signoise, 25e6)
# plot_constellation(signoise)

In [ ]:
signoise= impairments.apply_PMD(signoise, np.pi/5.5, 10e-12)
plot_constellation(signoise)

Match filtering

In [ ]:
signoise = signoise.resample(sig.fb*2, beta=0.1, renormalise=True)
plot_constellation(signoise)

linear compensation: 

- here we use a CMA equaliser with 17 taps and a stepsize 𝜇=2⋅10−3

In [ ]:
wxy, err = equalisation.equalise_signal(signoise, 2e-3, Ntaps=17)
sig_out = equalisation.apply_filter(signoise, wxy)
plot_constellation(sig_out)

In [ ]:
freq_offset = phaserec.find_freq_offset(sig_out,average_over_modes = False, fft_size = 2**15)
freq_offset
sig_out = phaserec.comp_freq_offset(sig_out,freq_offset)
plot_constellation(sig_out)

In [ ]:
sig_out, ph = phaserec.viterbiviterbi(sig_out, 11)
plot_constellation(sig_out)

Why do we need to get rid of the signal edges ?

In [ ]:
sig_out = helpers.dump_edges(sig_out, 20)
sig_out = helpers.normalise_and_center(sig_out)
plot_constellation(sig_out)

In [ ]:
ser = sig_out.cal_ser()
ber = sig_out.cal_ber()
print("SER_x = %.2f, SER_y= %.2f"%(ser[0], ser[1]))
print("BER_x = %.2f, BER_y = %.2f"%(ber[0], ber[1]))